<a href="https://www.kaggle.com/code/pariyawan02/simplest-with-tfidf?scriptVersionId=143531009" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import logging
import os
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset,load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from datasets import load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold, GroupKFold

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [ ]:
# Read all files from  
prompt_test = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv")
prompt_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv")
sample_submission = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv")
summaries_test = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv")
summaries_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv")

In [ ]:
summaries_train.head()

In [ ]:
prompt_train.head()

In [ ]:
summaries_train.shape

In [ ]:
# Getting Context of text files 
summaries_train['text'][0]

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
import string
ps = PorterStemmer()

In [ ]:
# Function for transformed text
stop_words = set(stopwords.words('english'))
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
# Text preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords and punctuation
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

In [ ]:
# Preprocess the summary data remove stopwords convert into lower form remove punctuations
summaries_train['processed_summaries_train'] = summaries_train['text'].apply(preprocess_text)

In [ ]:
prompt_train['processed_prompt_train'] = prompt_train['prompt_text'].apply(preprocess_text)

In [ ]:
prompt_train.head()

In [ ]:
train_data = pd.merge(summaries_train,prompt_train,on = "prompt_id")

In [ ]:
train_data.shape

In [ ]:
# TFIDF Vectorizer 
tfidf_vectorizer = TfidfVectorizer(max_features =500)
summaries_train_tf = tfidf_vectorizer.fit_transform(train_data['processed_summaries_train'])
prompt_train_tf = tfidf_vectorizer.fit_transform(train_data['processed_prompt_train'])

In [ ]:
# Concatenate the TF-IDF features
import scipy.sparse
features = scipy.sparse.hstack([prompt_train_tf,summaries_train_tf])

In [ ]:
features

In [ ]:
target_content = train_data['content']
target_wording = train_data['wording']

In [ ]:
target_content

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features,target_content, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create the Random Forest Regressor model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust hyperparameters as needed

# Train the model on the training data
rf_regressor.fit(X_train, y_train)
# prediction = rf_regressor.predict(X_test)

In [ ]:
# prediction


In [ ]:
from sklearn.model_selection import train_test_split
rf_regressor_1 = RandomForestRegressor(n_estimators=100, random_state=42)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(features,target_wording, test_size=0.2, random_state=42)
rf_regressor_1.fit(X_train_1,y_train_1)

In [ ]:
summaries_test['processed_summaries_test'] = summaries_test['text'].apply(preprocess_text)
prompt_test['processed_prompt_test'] = prompt_test['prompt_text'].apply(preprocess_text)
test_data = pd.merge(summaries_test,prompt_test,on = "prompt_id")

summaries_test_tf = tfidf_vectorizer.transform(test_data['processed_summaries_test'])
prompt_test_tf = tfidf_vectorizer.transform(test_data['processed_prompt_test'])

features_test = scipy.sparse.hstack([prompt_test_tf,summaries_test_tf])


In [ ]:
features_test

In [ ]:
prompt_test.shape

In [ ]:
predict = rf_regressor.predict(features_test)

In [ ]:
predict_1 = rf_regressor_1.predict(features_test)

In [ ]:
result = pd.DataFrame({
    'student_id': test_data['student_id'],
    'content': predict,
    'wording': predict_1
})

In [ ]:
# result.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
# result.to_csv('submission.csv',index = False)

In [ ]:
## Plz Upvote it If you found its helpful for You